<a href="https://colab.research.google.com/github/Yamada8412/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#辞書攻撃に用いられる辞書を用いて、パスワード強度判定プログラムを作る。
パスワードの強度の判定結果は、物凄く弱い、弱い、中ぐらい、強い、物凄く強いのどれかである。

In [96]:
import pandas as pd

calc_len関数は、パスワードの桁数に応じて、そのパスワードの強度を評価する。
パスワードの桁数が４桁以下だと0を返し、５桁以上だとその桁に応じて評価する。

In [145]:
def calc_len(p):
  #pはpassword
  if len(p) <= 4:
    return 0
  else:
    return ((len(p) - 1) * 2)
p = input()
calc_len(p)

123


TypeError: ignored

calc_sca_kinds関数は、パスワードの散らばり具合(scatter)と、種類(kinds)を調べる関数。文字の種類は　[大文字、小文字、数字、記号]　のどれかで判別する。

In [115]:
def calc_sca_kinds(p):
  #pはpassword
  #パスワードをlistの中に入れて、１文字ずつに分解
  p_list = list(p)

  up_al = list() #upper_alphabet(大文字)
  un_al = list() #under_alphabet(小文字)
  num   = list() #number(数字)
  sym   = list() #symbol(記号)

  #それぞれのリストに、一致した文字の"インデックス"を追加する
  for i in range(len(p_list)):
    if p_list[i].isnumeric():
      num.append(i)
    elif p_list[i].isupper():
      up_al.append(i)
    elif (p_list[i].islower()):
      un_al.append(i)
    else:
      sym.append(i)

  scatter_list = [0] * 4
  kinds_cnt = 0

  p_name_list = [up_al,un_al,num,sym] #上で集計したリストを[大英字、小英字、数字、記号]の順に格納

  for i in p_name_list:# 文字が4種類中、何種類あるか数える
    if len(i) >= 1:
      kinds_cnt += 1

  for i in range(4):
    a = 0
    for x in range(len(p_name_list[i]) - 1):
      a += (p_name_list[i][x + 1] - p_name_list[i][x])
    if len(p_name_list[i]) == 0:
      pass
    elif (a - len(p_name_list[i]) + 1) >= 1:
      scatter_list[i] = a - len(p_name_list[i]) + 1

  return kinds_cnt,scatter_list


上のcacl_sca_kinds関数の返り値は、例えば 

Suzuki1224だと、文字の種類は3種類でS,uzuki,1224がそれぞれ束になっているので >> 3,[0,0,0,0]

Su12zuki24だと、12と24,uとzukiがそれぞれ離れており、大文字は1文字、記号は0文字なので>> 3,[0,2,4,0]   
 
A!6wは4種類の文字が含まれているが、それぞれ1つずつしかないので >> 4,[0,0,0,0]

※それぞれ[大文字、小文字、数字、記号]の離れ具合を表す。


下のmake_dictionary関数は、よく使われる危険なパスワードをhttps://s1.nordcdn.com/nord/misc/0.55.0/nordpass/200-most-common-passwords-en.pdf
から引用しcsvファイルにした表を、2019年、2020年、2021年の3つのデータがある状態から、評価しやすいように1つにまとめる関数



上のurlにあるデータをまとめたものです

In [134]:
df = pd.read_csv('pass_str.csv') 
df

,2019 password,2019 count,2020 password,2020 count,2021 password,2021 count
0,12345,2812220,123456,2543285,123456,103170552
1,123456,2485216,123456789,961435,123456789,46027530
2,123456789,1052268,picture1,371612,12345,32955431
3,test1,993756,password,360467,qwerty,22317280
4,password,830846,12345678,322187,password,20958297
...,...,...,...,...,...,...
195,123456789,29110,142536,15895,1234567891,591221
196,a123456,29103,peanut,15832,456123,589201
197,samsung,29073,11223344,15830,444444,586266
198,123,29068,thomas,15817,qq123456,558240


In [138]:
def make_dicti():
  #'pass_str.csvを読み込み 
  df = pd.read_csv('pass_str.csv')

  #それぞれの年で'pass_str.csv'の中にあるパスワードを使っている人の母数が違うため、
  #2019,2020,2021年における、あるパスワードが使われている数をその年に調査された人数全体の割合で表す。
  for i in range(2019,2022):
    df[f'{i} count'] = df[f'{i} count'] / df[f'{i} count'].sum()

  #df_returnを定義　　2021年のパスワードと使用人数をdf_returnに移す。コラムをそれぞれ変更
  df_return = df[['2021 password','2021 count']]
  df_return = df_return.rename(columns = {'2021 password': 'password','2021 count':'count'})

  for i in range(2019,2021):
    #df_return_subを定義  df_returnと縦に結合するためにコラムを変更
    df_return_sub = df[[f'{i} password']]
    df_return_sub = df_return_sub.rename(columns = {f'{i} password':'password'})
    df_return_sub = df_return_sub.assign(count = 0)

    #df_returnの下にdf_return_subを結合
    df_return = df_return.append(df_return_sub,ignore_index = True)
    df_return = df_return[df_return.duplicated(subset = 'password',keep = 'first') == False]
    df_return = df_return.reset_index()

    #df_returnに入っているパスワードを上から一つずつ読み取り、2019,2020年のパスワードと比較
    #もし同じパスワードが見つかったら、そのパスワードの2019年、2020年度における使用人数を2で割って足す
    #2で割る理由は、2021年のデータの方が新しく、信用できるため、2019,2020年のデータが2021年のデータに、大きく影響しないようにするため。
    for x in range(len(df_return)):
      try:
        a = df[df_return.loc[x,'password'] == df[f'{i} password']][f'{i} count'].sum()
        df_return.loc[x,'count'] += float(a)/2
      except:
        pass
  return df_return.sort_values('count',ascending = False)

make_dictiの中身です。　　列名"level_0"と"index"は、reset_indexをすると自然に出てくるので無視してください

In [139]:
make_dicti() 

,level_0,index,password,count
0,0,0.0,123456,0.349529
1,1,1.0,123456789,0.148986
2,2,2.0,12345,0.133384
4,4,4.0,password,0.071980
3,3,3.0,qwerty,0.053722
...,...,...,...,...
292,292,393.0,jonathan,0.000713
328,479,NaN,333333,0.000710
329,489,NaN,142536,0.000691
330,493,NaN,angel1,0.000687


In [ ]:
def calc_dict(dicti,p):
  if dicti['password'] == p:
    return 'a'


In [143]:
p = input('ここにパスワードを入力してください：')

len = calc_len(p)
scater = calc_sca_kinds(p)[0]
kinds = calc_sca_kinds(p)[1]
dictio = make_dicti()

calc_dict(dictio,p)


ここにパスワードを入力してください：Suzuki1224


TypeError: ignored